# Next steps 

In this file, I am going to the use the following files: 
- action_units_temp_456_extracted: dataframe, has all of the information in it (TSfresh)
- VVR_scores_final: this has the questionnaire data

I need to merge them to be able to apply a model, such as random forest. 

In [1]:
# import 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Import the files we need 

In [2]:
VVR_scores_final = pd.read_csv('/Users/dionnespaltman/Desktop/V3/VVR_scores_final.csv', sep=',')
clean_donor_info = pd.read_csv('/Users/dionnespaltman/Desktop/V3/processed_donor_info.csv', sep=',')

VVR_scores_final.drop('Unnamed: 0', axis=1, inplace=True)

clean_donor_info.drop('Unnamed: 0.1', axis=1, inplace=True)
clean_donor_info.drop('Unnamed: 0', axis=1, inplace=True)
clean_donor_info.drop('Index', axis=1, inplace=True)

display(VVR_scores_final.head(5))
display(clean_donor_info.head(5))

,Unnamed: 0.1,ID,sum_12,sum_4567,sum_456,VVR_group,Condition,VVR_1,VVR_2
0,0,5,22.0,34.0,26.0,0,1,12.0,10.0
1,1,6,16.0,32.0,24.0,0,2,8.0,8.0
2,2,7,23.0,77.0,65.0,1,2,11.0,12.0
3,3,8,17.0,32.0,24.0,0,1,9.0,8.0
4,4,9,16.0,32.0,24.0,0,1,8.0,8.0


,Unnamed: 0.2,ID,Stage,Gender,Age,Date,Location,Condition,Weight,Length,...,Weakness,Lightheadedness,Fear,Tension,Stress,Nervousness,Physical_sum,Psychological_sum,na.rm,sum_VVR
0,0,5,1.0,2,33.0,2019-10-21,0,1,74.0,171.0,...,1.0,1.0,2.0,2.0,2.0,2.0,4.0,8.0,True,12.0
1,1,5,2.0,2,33.0,2019-10-21,0,1,74.0,171.0,...,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0,True,10.0
2,2,5,3.0,2,33.0,2019-10-21,0,1,74.0,171.0,...,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0,True,10.0
3,3,5,4.0,2,33.0,2019-10-21,0,1,74.0,171.0,...,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0,True,9.0
4,4,5,5.0,2,33.0,2019-10-21,0,1,74.0,171.0,...,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0,True,9.0


In [3]:
extracted_features = pd.read_csv('/Users/dionnespaltman/Desktop/V3/action_units_temp_456_extracted.csv', sep=',')
extracted_features.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
display(extracted_features)

,ID,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__minimum,AU45_r__mean,AU45_r__mean_abs_change,AU01_r__sum_values,AU01_r__variance,...,AU25_r__minimum,AU25_r__mean,AU25_r__mean_abs_change,AU26_r__sum_values,AU26_r__variance,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change
0,23,9231.74,0.825039,0.908316,4.91,0.00,0.627753,0.133624,4982.48,0.425041,...,0.00,0.467260,0.067429,9313.07,0.511950,0.715507,5.00,0.00,0.633284,0.076328
1,24,11887.00,0.634554,0.796589,5.00,0.00,0.436942,0.098134,9390.23,0.448366,...,0.00,1.125899,0.105669,40391.28,1.977784,1.406337,5.00,0.00,1.484701,0.125851
2,25,9020.78,0.750701,0.866430,4.04,0.00,0.550652,0.085720,6954.35,0.599805,...,0.00,0.557403,0.082317,14126.22,0.993081,0.996534,5.00,0.00,0.862301,0.101969
3,26,6585.31,0.609348,0.780607,4.90,0.00,0.371673,0.056287,9707.43,0.873280,...,0.00,0.457041,0.056427,9786.70,0.463418,0.680748,5.00,0.00,0.552359,0.069582
4,27,23027.73,1.160635,1.077328,5.04,-4.29,1.094318,0.231853,21049.90,1.475421,...,-3.11,0.526013,0.285330,2988.67,1.876089,1.369704,6.69,-3.92,0.142027,0.386527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,142,10547.03,0.506052,0.711374,3.53,0.00,0.457691,0.055759,11750.48,0.739290,...,0.00,0.808731,0.094557,10323.87,0.279436,0.528617,5.00,0.00,0.448007,0.076771
107,143,6782.20,0.152370,0.390346,2.81,0.00,0.237997,0.035817,13978.96,0.763932,...,0.00,0.685215,0.115276,27360.64,1.239602,1.113374,5.00,0.00,0.960124,0.142141
108,144,7319.31,0.418456,0.646882,3.78,0.00,0.391344,0.072455,7175.00,0.461053,...,0.00,0.495268,0.072572,10309.28,0.389152,0.623820,5.00,0.00,0.551210,0.075307
109,145,16768.57,0.748187,0.864978,4.31,0.00,0.535566,0.118258,11354.47,0.516678,...,0.00,1.720578,0.102174,20078.78,0.423006,0.650389,4.89,0.00,0.641290,0.064991


In [4]:
# Making a list of 5 to 332 to check which are missing 
number_list = list(range(5, 333))

# Extract the 'ID' column from the donor_info DataFrame
id_column = VVR_scores_final['ID'].tolist()

# Find the IDs that are in number_list but not in id_column
missing_ids = [id_num for id_num in number_list if id_num not in id_column]

# Print the list of missing IDs
print("IDs in number_list but not in donor_info:", missing_ids)

IDs in number_list but not in donor_info: [11, 20, 55, 56, 70, 89, 105, 110]


In [5]:
print(len(extracted_features.columns))

120


# Merging files 

In [9]:
# Assuming 'ID' is the common column in both DataFrames
merged_df = pd.merge(VVR_scores_final, extracted_features, on='ID', how='inner')

# Check the length of the merged DataFrame
print(len(merged_df))
# merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)
# merged_df.drop('Unnamed: 0', axis=1, inplace=True)
# merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)

display(merged_df)

111


,ID,sum_12,sum_4567,sum_456,VVR_group,Condition,VVR_1,VVR_2,AU45_r__sum_values,AU45_r__variance,...,AU25_r__minimum,AU25_r__mean,AU25_r__mean_abs_change,AU26_r__sum_values,AU26_r__variance,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change
0,23,24.0,37.0,27.0,0,2,13.0,11.0,9231.74,0.825039,...,0.00,0.467260,0.067429,9313.07,0.511950,0.715507,5.00,0.00,0.633284,0.076328
1,24,23.0,37.0,28.0,0,2,12.0,11.0,11887.00,0.634554,...,0.00,1.125899,0.105669,40391.28,1.977784,1.406337,5.00,0.00,1.484701,0.125851
2,25,28.0,44.0,33.0,1,2,16.0,12.0,9020.78,0.750701,...,0.00,0.557403,0.082317,14126.22,0.993081,0.996534,5.00,0.00,0.862301,0.101969
3,26,30.0,37.0,29.0,0,1,15.0,15.0,6585.31,0.609348,...,0.00,0.457041,0.056427,9786.70,0.463418,0.680748,5.00,0.00,0.552359,0.069582
4,27,22.0,39.0,31.0,1,2,11.0,11.0,23027.73,1.160635,...,-3.11,0.526013,0.285330,2988.67,1.876089,1.369704,6.69,-3.92,0.142027,0.386527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,142,20.0,34.0,26.0,0,3,11.0,9.0,10547.03,0.506052,...,0.00,0.808731,0.094557,10323.87,0.279436,0.528617,5.00,0.00,0.448007,0.076771
107,143,25.0,48.0,36.0,1,3,14.0,11.0,6782.20,0.152370,...,0.00,0.685215,0.115276,27360.64,1.239602,1.113374,5.00,0.00,0.960124,0.142141
108,144,24.0,35.0,27.0,0,3,12.0,12.0,7319.31,0.418456,...,0.00,0.495268,0.072572,10309.28,0.389152,0.623820,5.00,0.00,0.551210,0.075307
109,145,20.0,37.0,28.0,0,1,11.0,9.0,16768.57,0.748187,...,0.00,1.720578,0.102174,20078.78,0.423006,0.650389,4.89,0.00,0.641290,0.064991


In [10]:
merged_df.to_csv('/Users/dionnespaltman/Desktop/V3/merged_df.csv', index=True)


In [46]:
# Count the number of NaN values in each column of the DataFrame
nan_counts = merged_df.isna().sum()

# Print columns with NaN values
nan_columns = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(nan_columns)

Columns with NaN values:
Series([], dtype: int64)


# Descriptives 

In [47]:
display(merged_df)

,Unnamed: 0.1,ID,sum_12,sum_4567,sum_456,VVR_group,Condition,VVR_1,VVR_2,AU01_r__sum_values,...,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__minimum,AU45_r__mean,AU45_r__mean_abs_change
0,16,23,24.0,37.0,27.0,0,2,13.0,11.0,4982.48,...,0.00,0.633284,0.076328,9231.74,0.825039,0.908316,4.91,0.00,0.627753,0.133624
1,17,24,23.0,37.0,28.0,0,2,12.0,11.0,9390.23,...,0.00,1.484701,0.125851,11887.00,0.634554,0.796589,5.00,0.00,0.436942,0.098134
2,18,25,28.0,44.0,33.0,1,2,16.0,12.0,6954.35,...,0.00,0.862301,0.101969,9020.78,0.750701,0.866430,4.04,0.00,0.550652,0.085720
3,19,26,30.0,37.0,29.0,0,1,15.0,15.0,9707.43,...,0.00,0.552359,0.069582,6585.31,0.609348,0.780607,4.90,0.00,0.371673,0.056287
4,20,27,22.0,39.0,31.0,1,2,11.0,11.0,21049.90,...,-3.92,0.142027,0.386527,23027.73,1.160635,1.077328,5.04,-4.29,1.094318,0.231853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,129,142,20.0,34.0,26.0,0,3,11.0,9.0,11750.48,...,0.00,0.448007,0.076771,10547.03,0.506052,0.711374,3.53,0.00,0.457691,0.055759
107,130,143,25.0,48.0,36.0,1,3,14.0,11.0,13978.96,...,0.00,0.960124,0.142141,6782.20,0.152370,0.390346,2.81,0.00,0.237997,0.035817
108,131,144,24.0,35.0,27.0,0,3,12.0,12.0,7175.00,...,0.00,0.551210,0.075307,7319.31,0.418456,0.646882,3.78,0.00,0.391344,0.072455
109,132,145,20.0,37.0,28.0,0,1,11.0,9.0,11354.47,...,0.00,0.641290,0.064991,16768.57,0.748187,0.864978,4.31,0.00,0.535566,0.118258


In [19]:
columns = list(merged_df.columns)
print(columns)

['ID', 'sum_12', 'sum_4567', 'sum_456', 'VVR_group', 'Condition', 'VVR_1', 'VVR_2', 'AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__minimum', 'AU01_r__mean', 'AU01_r__mean_abs_change', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__minimum', 'AU02_r__mean', 'AU02_r__mean_abs_change', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__minimum', 'AU04_r__mean', 'AU04_r__mean_abs_change', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__minimum', 'AU05_r__mean', 'AU05_r__mean_abs_change', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__minimum', 'AU06_r__mean', 'AU06_r__mean_abs_change', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__minimum', 'AU07_r__mean', 'AU07_r__mean_abs_change'

In [22]:
columns_au = list(extracted_features.columns)
print(columns_au)

columns_action_units = ['AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__minimum', 'AU01_r__mean', 'AU01_r__mean_abs_change', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__minimum', 'AU02_r__mean', 'AU02_r__mean_abs_change', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__minimum', 'AU04_r__mean', 'AU04_r__mean_abs_change', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__minimum', 'AU05_r__mean', 'AU05_r__mean_abs_change', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__minimum', 'AU06_r__mean', 'AU06_r__mean_abs_change', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__minimum', 'AU07_r__mean', 'AU07_r__mean_abs_change', 'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09_r__maximum', 'AU09_r__minimum', 'AU09_r__mean', 'AU09_r__mean_abs_change', 'AU10_r__sum_values', 'AU10_r__variance', 'AU10_r__standard_deviation', 'AU10_r__maximum', 'AU10_r__minimum', 'AU10_r__mean', 'AU10_r__mean_abs_change', 'AU12_r__sum_values', 'AU12_r__variance', 'AU12_r__standard_deviation', 'AU12_r__maximum', 'AU12_r__minimum', 'AU12_r__mean', 'AU12_r__mean_abs_change', 'AU14_r__sum_values', 'AU14_r__variance', 'AU14_r__standard_deviation', 'AU14_r__maximum', 'AU14_r__minimum', 'AU14_r__mean', 'AU14_r__mean_abs_change', 'AU15_r__sum_values', 'AU15_r__variance', 'AU15_r__standard_deviation', 'AU15_r__maximum', 'AU15_r__minimum', 'AU15_r__mean', 'AU15_r__mean_abs_change', 'AU17_r__sum_values', 'AU17_r__variance', 'AU17_r__standard_deviation', 'AU17_r__maximum', 'AU17_r__minimum', 'AU17_r__mean', 'AU17_r__mean_abs_change', 'AU20_r__sum_values', 'AU20_r__variance', 'AU20_r__standard_deviation', 'AU20_r__maximum', 'AU20_r__minimum', 'AU20_r__mean', 'AU20_r__mean_abs_change', 'AU23_r__sum_values', 'AU23_r__variance', 'AU23_r__standard_deviation', 'AU23_r__maximum', 'AU23_r__minimum', 'AU23_r__mean', 'AU23_r__mean_abs_change', 'AU25_r__sum_values', 'AU25_r__variance', 'AU25_r__standard_deviation', 'AU25_r__maximum', 'AU25_r__minimum', 'AU25_r__mean', 'AU25_r__mean_abs_change', 'AU26_r__sum_values', 'AU26_r__variance', 'AU26_r__standard_deviation', 'AU26_r__maximum', 'AU26_r__minimum', 'AU26_r__mean', 'AU26_r__mean_abs_change', 'AU45_r__sum_values', 'AU45_r__variance', 'AU45_r__standard_deviation', 'AU45_r__maximum', 'AU45_r__minimum', 'AU45_r__mean', 'AU45_r__mean_abs_change']

['ID', 'AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__minimum', 'AU01_r__mean', 'AU01_r__mean_abs_change', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__minimum', 'AU02_r__mean', 'AU02_r__mean_abs_change', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__minimum', 'AU04_r__mean', 'AU04_r__mean_abs_change', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__minimum', 'AU05_r__mean', 'AU05_r__mean_abs_change', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__minimum', 'AU06_r__mean', 'AU06_r__mean_abs_change', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__minimum', 'AU07_r__mean', 'AU07_r__mean_abs_change', 'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU

In [24]:
# import json

# # Save columns_action_units list to a JSON file
# with open('/Users/dionnespaltman/Desktop/V3/columns_action_units.json', 'w') as f:
#     json.dump(columns_action_units, f)

# Descriptives about dataset 